In [1]:
import sqlite3
import pandas as pd

In [3]:
# Create / connect to database
conn = sqlite3.connect("../subscription.db")

In [5]:
customers = pd.read_csv("../data/users.csv")

customers.to_sql(
    name="users",
    con=conn,
    if_exists="replace",
    index=False
)

10300

In [6]:
users = pd.read_csv("../data/users.csv")
users.to_sql("users", conn, if_exists="replace", index=False)

10300

In [7]:
movies = pd.read_csv("../data/movies.csv")
movies.to_sql("movies", conn, if_exists="replace", index=False)

1040

In [8]:
watch_history = pd.read_csv("../data/watch_history.csv")
watch_history.to_sql("watch_history", conn, if_exists="replace", index=False)

105000

In [9]:
reviews = pd.read_csv("../data/reviews.csv")
reviews.to_sql("reviews", conn, if_exists="replace", index=False)

15450

In [10]:
search_logs = pd.read_csv("../data/search_logs.csv")
search_logs.to_sql("search_logs", conn, if_exists="replace", index=False)

26500

In [11]:
recommendation_logs = pd.read_csv("../data/recommendation_logs.csv")
recommendation_logs.to_sql("recommendation_logs", conn, if_exists="replace", index=False)

52000

In [12]:
pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)

,name
0,users
1,movies
2,watch_history
3,reviews
4,search_logs
5,recommendation_logs


In [17]:
watch_query = """
SELECT
    user_id,
    COUNT(*) AS total_watches,
    COUNT(DISTINCT DATE(watch_date)) AS active_days,
    MAX(watch_date) AS last_watch_date
FROM watch_history
GROUP BY user_id;
"""

watch_summary = pd.read_sql_query(watch_query, conn)

watch_summary.to_sql(
    "watch_summary",
    conn,
    if_exists="replace",
    index=False
)

watch_summary.head()

,user_id,total_watches,active_days,last_watch_date
0,user_00001,13,12,2025-10-30
1,user_00002,15,14,2025-11-28
2,user_00003,9,8,2025-06-24
3,user_00004,15,14,2025-12-29
4,user_00005,10,9,2025-09-19


In [18]:
reviews_query = """
SELECT
    user_id,
    COUNT(*) AS total_reviews,
    AVG(rating) AS avg_rating
FROM reviews
GROUP BY user_id;
"""

reviews_summary = pd.read_sql_query(reviews_query, conn)

reviews_summary.to_sql(
    "reviews_summary",
    conn,
    if_exists="replace",
    index=False
)

reviews_summary.head()

,user_id,total_reviews,avg_rating
0,user_00001,1,4.000000
1,user_00002,4,3.500000
2,user_00003,1,4.000000
3,user_00004,4,2.250000
4,user_00005,3,3.333333


In [19]:
search_query = """
SELECT
    user_id,
    COUNT(*) AS total_searches,
    MAX(search_date) AS last_search_date
FROM search_logs
GROUP BY user_id;
"""

search_summary = pd.read_sql_query(search_query, conn)

search_summary.to_sql(
    "search_summary",
    conn,
    if_exists="replace",
    index=False
)

search_summary.head()

,user_id,total_searches,last_search_date
0,user_00001,1,2024-12-29
1,user_00002,3,2025-12-08
2,user_00004,2,2025-12-29
3,user_00005,2,2025-09-29
4,user_00006,1,2024-06-11


In [21]:
recommendation_query = """
SELECT
    user_id,
    COUNT(*) AS total_recommendations
FROM recommendation_logs
GROUP BY user_id;
"""

recommendation_summary = pd.read_sql_query(recommendation_query, conn)

recommendation_summary.to_sql(
    "recommendation_summary",
    conn,
    if_exists="replace",
    index=False
)

recommendation_summary.head()

,user_id,total_recommendations
0,user_00001,3
1,user_00003,6
2,user_00004,4
3,user_00005,1
4,user_00006,8


In [24]:
pd.read_sql_query(
    "PRAGMA table_info(users);",
    conn
)

,cid,name,type,notnull,dflt_value,pk
0,0,user_id,TEXT,0,None,0
1,1,email,TEXT,0,None,0
2,2,first_name,TEXT,0,None,0
3,3,last_name,TEXT,0,None,0
4,4,age,REAL,0,None,0
5,5,gender,TEXT,0,None,0
6,6,country,TEXT,0,None,0
7,7,state_province,TEXT,0,None,0
8,8,city,TEXT,0,None,0
9,9,subscription_plan,TEXT,0,None,0


In [25]:
final_query = """
SELECT
    u.user_id,
    u.country,
    u.subscription_plan,
    u.subscription_start_date,
    u.monthly_spend,
    u.is_active,
    u.primary_device,
    u.created_at,

    ws.total_watches,
    ws.active_days,
    ws.last_watch_date,

    rs.total_reviews,
    rs.avg_rating,

    ss.total_searches,
    ss.last_search_date,

    rl.total_recommendations

FROM users u
LEFT JOIN watch_summary ws
    ON u.user_id = ws.user_id
LEFT JOIN reviews_summary rs
    ON u.user_id = rs.user_id
LEFT JOIN search_summary ss
    ON u.user_id = ss.user_id
LEFT JOIN recommendation_summary rl
    ON u.user_id = rl.user_id;
"""

In [26]:
user_level_master = pd.read_sql_query(final_query, conn)
user_level_master.head()

,user_id,country,subscription_plan,subscription_start_date,monthly_spend,is_active,primary_device,created_at,total_watches,active_days,last_watch_date,total_reviews,avg_rating,total_searches,last_search_date,total_recommendations
0,user_00001,USA,Basic,2024-04-08,36.06,1,Laptop,2023-04-01 14:40:50.540242,13,12,2025-10-30,1.0,4.000000,1.0,2024-12-29,3.0
1,user_00002,USA,Premium+,2024-05-24,14.59,1,Desktop,2024-10-10 15:39:11.030515,15,14,2025-11-28,4.0,3.500000,3.0,2025-12-08,NaN
2,user_00003,USA,Standard,2023-09-22,11.71,0,Desktop,2024-06-29 14:27:49.560875,9,8,2025-06-24,1.0,4.000000,NaN,None,6.0
3,user_00004,USA,Standard,2024-08-21,28.56,1,Laptop,2023-04-11 01:01:59.614841,15,14,2025-12-29,4.0,2.250000,2.0,2025-12-29,4.0
4,user_00005,USA,Standard,2024-10-28,9.54,1,Desktop,2025-04-12 19:59:30.137806,10,9,2025-09-19,3.0,3.333333,2.0,2025-09-29,1.0


In [27]:
user_level_master.to_csv(
    "../data/user_level_master.csv",
    index=False
)

In [28]:
user_level_master.shape

(10300, 16)